In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.191, 0.081, 0.067, 0.111],
    [0.082, 0.059, 0.063, 0.102],
    [0.036, 0.083, 0.076, 0.088],
    [0.029, 0.088, 0.235, 0.078],
    [0.306, 0.055, 0.113, 0.068],
    [0.038, 0.154, 0.033, 0.144],
    [0.014, 0.100, 0.047, 0.148],
    [0.029, 0.155, 0.139, 0.105],
    [0.014, 0.083, 0.150, 0.089], 
    [0.010, 0.142, 0.067, 0.069]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

def marcoss(matrix, weights, criteria_types, sites):
    # Step 1: Define AI and AAI
    ai = np.where(np.array(criteria_types) == 'max', 
                  np.max(matrix, axis=0), 
                  np.min(matrix, axis=0))
    
    aai = np.where(np.array(criteria_types) == 'max', 
                   np.min(matrix, axis=0), 
                   np.max(matrix, axis=0))
    
    # Step 2: Extend matrix with AI and AAI
    extended = np.vstack([matrix, ai, aai])

    # Step 3: Normalize extended matrix
    norm_extended = np.zeros_like(extended, dtype=float)
    for j in range(extended.shape[1]):
        if criteria_types[j] == 'max':
            norm_extended[:, j] = extended[:, j] / ai[j]
        else:
            norm_extended[:, j] = aai[j] / extended[:, j]
    
    # Step 4: Apply weights to normalized matrix
    weighted_extended = norm_extended * weights

    # Step 5: Compute S_i, S_AI, S_AAI
    S = weighted_extended.sum(axis=1)
    S_ai, S_aai = S[-2], S[-1]  # Last two rows are AI and AAI
    S_alts = S[:-2]             # Alternatives only

    # Step 6: Compute utility degrees
    K_plus = S_alts / S_ai
    K_minus = S_alts / S_aai

    # Step 7: Final utility function as compromise score
    utility = (K_plus + K_minus) / (
        1 + (1 - K_plus) / K_plus + (1 - K_minus) / K_minus
    )

    # Step 8: Rank alternatives
    ranks = rankdata(-utility, method='min')

    # Step 9: Return results
    return pd.DataFrame({
        'Site': sites,
        'Utility': utility,
        'Rank': ranks
    }).sort_values(by='Rank').reset_index(drop=True)

# Run MARCOS and show result
result_df = marcoss(matrix, weights, criteria_types, sites)
print(result_df)

     Site   Utility  Rank
0   Site4  5.200104     1
1   Site8  4.128421     2
2   Site5  2.334737     3
3   Site9  2.111605     4
4   Site6  1.564461     5
5   Site1  1.498939     6
6  Site10  1.481736     7
7   Site3  0.955989     8
8   Site7  0.921082     9
9   Site2  0.728980    10
